## 1. Instalação e Setup

In [6]:
# YOLO + StrongSORT + OpenCV + Supervision
!pip install ultralytics supervision opencv-python --quiet



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importação das bibliotecas necessárias
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## 2. Carregando um modelo pré-treinado

In [ ]:
# Carregando o modelo YOLOv8n (nano - versão mais leve)
model = YOLO('yolov8l.pt')

# Também podemos carregar diferentes tamanhos de modelo:
# model = YOLO('yolov8s.pt')  # small
# model = YOLO('yolov8m.pt')  # medium
# model = YOLO('yolov8l.pt')  # large
# model = YOLO('yolov8x.pt')  # extra large

# Ativando o StrongSORT como rastreador
model.trackers = "botsort.yaml"  # pode trocar por 'strongsort.yaml' se preferir

## 4. Função para processar vídeo e rastrear motos com IDs

In [ ]:
def track_motos(video_path):
    cap = cv2.VideoCapture(video_path)

    frames_annotated = []
    ids_inicio = set()
    ids_fim = set()

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idx = 0

    print("⏳ Iniciando rastreamento do vídeo...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detecção com rastreamento (apenas motos: classe 3 no COCO)
        results = model.track(
            frame,
            persist=True,
            conf=0.5,
            iou=0.7,
            classes=[3],  # somente motos
            verbose=False
        )

        # Desenhar resultados no frame
        annotated_frame = results[0].plot()
        frames_annotated.append(annotated_frame)

        # Extrair IDs rastreados neste frame
        boxes = results[0].boxes
        ids = boxes.id if boxes and boxes.id is not None else []

        # Armazena os IDs do primeiro e último frame
        if frame_idx == 0:
            ids_inicio.update([int(i) for i in ids])
        elif frame_idx == frame_count - 1:
            ids_fim.update([int(i) for i in ids])

        frame_idx += 1

    cap.release()

    # Análise final
    motos_estacionadas = ids_inicio & ids_fim
    motos_sairam = ids_inicio - ids_fim
    motos_entraram = ids_fim - ids_inicio

    print("\nResumo do rastreamento:")
    print(f"Motos que permaneceram estacionadas: {motos_estacionadas}")
    print(f"Motos que saíram: {motos_sairam}")
    print(f"Motos que entraram: {motos_entraram}")

    return frames_annotated, motos_entraram, motos_sairam, motos_estacionadas


## 5. Função para visualizar um frame específico com as detecções

In [ ]:
def show_frame(frames, index):
    if index < 0:
        index = len(frames) + index  # permite usar -1 para último frame
    if 0 <= index < len(frames):
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(frames[index], cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Frame {index}')
        plt.show()
    else:
        print("❌ Índice fora do intervalo.")


## 6. Enviar vídeo e executar o rastreamento

In [ ]:
# Substitua pelo nome do seu vídeo ou caminho completo
video_path = "video.mp4"  # Exemplo: "estacionamento_motttu.mp4"

# Executa rastreamento com StrongSORT
frames, entrou, saiu, permaneceu = track_motos(video_path)

## 7. Visualizar frames anotados com IDs

In [ ]:
# Primeiro frame com IDs
show_frame(frames, 0)

# Último frame com IDs
show_frame(frames, -1)


## 8. (Opcional) Salvar vídeo final com rastreamento

In [ ]:
def export_video(frames, output_path, fps=30):
    h, w, _ = frames[0].shape
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    for frame in frames:
        writer.write(frame)
    writer.release()

    print(f"🎬 Vídeo exportado com sucesso para: {output_path}")


In [ ]:
export_video(frames, "saida_video_com_rastreamento.mp4")


##  7. Lista de classes COCO

In [16]:
# Lista de todas as classes que o modelo reconhece
for idx, class_name in model.names.items():
    print(f"{idx}: {class_name}")


0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush
